In [1]:
print("""
۱. چرا داده‌ها را به chunk تبدیل می‌کنیم؟ تأثیر chunk_size و chunk_overlap چیست؟
تقسیم داده‌ها به chunk باعث پردازش بهتر متن و استخراج دقیق‌تر اطلاعات توسط مدل می‌شود. chunk_size اندازه هر بخش را تعیین می‌کند و chunk_overlap با ایجاد همپوشانی، از گسستگی اطلاعات جلوگیری می‌کند.

۲. چرا بهتر است که بازیابی ترکیبی باشد؟ بازیابی ترکیبی به چه معنی است؟
بازیابی ترکیبی از دو روش جستجوی کلمات کلیدی و جستجوی معنایی استفاده می‌کند تا اطلاعات دقیق‌تری ارائه دهد. این روش باعث افزایش دقت چت‌بات در یافتن پاسخ‌های مرتبط می‌شود.

۳. چرا قبل از تولید پاسخ، context‌های بازیابی یا جستجو شده را فیلتر می‌کنیم؟
فیلترگذاری context باعث حذف اطلاعات نامرتبط و افزایش دقت پاسخ می‌شود. این کار از ایجاد پاسخ‌های نادرست توسط مدل جلوگیری می‌کند.

۴. مدل قابلیت پاسخ به سوالات را دارد. چرا کلاً این وظیفه را به مدل نمی‌سپاریم و RAG پیاده‌سازی کردیم؟
مدل‌های زبانی ممکن است اطلاعات نادرست تولید کنند، اما RAG با ترکیب جستجو و تولید متن، پاسخ‌های دقیق‌تری ارائه می‌دهد. این روش از داده‌های به‌روز استفاده کرده و اعتبار پاسخ‌ها را افزایش می‌دهد.
""")


۱. چرا داده‌ها را به chunk تبدیل می‌کنیم؟ تأثیر chunk_size و chunk_overlap چیست؟
تقسیم داده‌ها به chunk باعث پردازش بهتر متن و استخراج دقیق‌تر اطلاعات توسط مدل می‌شود. chunk_size اندازه هر بخش را تعیین می‌کند و chunk_overlap با ایجاد همپوشانی، از گسستگی اطلاعات جلوگیری می‌کند.

۲. چرا بهتر است که بازیابی ترکیبی باشد؟ بازیابی ترکیبی به چه معنی است؟
بازیابی ترکیبی از دو روش جستجوی کلمات کلیدی و جستجوی معنایی استفاده می‌کند تا اطلاعات دقیق‌تری ارائه دهد. این روش باعث افزایش دقت چت‌بات در یافتن پاسخ‌های مرتبط می‌شود.

۳. چرا قبل از تولید پاسخ، context‌های بازیابی یا جستجو شده را فیلتر می‌کنیم؟
فیلترگذاری context باعث حذف اطلاعات نامرتبط و افزایش دقت پاسخ می‌شود. این کار از ایجاد پاسخ‌های نادرست توسط مدل جلوگیری می‌کند.

۴. مدل قابلیت پاسخ به سوالات را دارد. چرا کلاً این وظیفه را به مدل نمی‌سپاریم و RAG پیاده‌سازی کردیم؟
مدل‌های زبانی ممکن است اطلاعات نادرست تولید کنند، اما RAG با ترکیب جستجو و تولید متن، پاسخ‌های دقیق‌تری ارائه می‌دهد. این روش از داده‌های به‌روز استفاده کرده و اعتبار پاسخ

In [9]:
import sqlite3
import os
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import LanceDB
import lancedb
from langchain.chains import RetrievalQA
from llama_parse import LlamaParse
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# تنظیمات API
AVALAI_BASE_URL = "https://api.avalai.ir/v1"
GPT_MODEL_NAME = "gpt-4o-mini"
GEMINI_MODEL_NAME = "gemini-1.5-flash"
TAVILY_API_KEY = "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"

# مقداردهی اولیه مدل‌ها
gpt4o_chat = ChatOpenAI(
    model=GPT_MODEL_NAME,
    base_url=AVALAI_BASE_URL,
    api_key="aa-IvpAOjBtYPok34GinJJjP88e7sfDmFni7ZFIbhxMrpa8P0c9",
)
LLAMA_CLOUD_API_KEY = "llx-tDiq3pq1fwKcrJX6adgMOS2SiTOvxDDFuD6HQ2jLTJ4fEMFc"
# gemini_chat = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME)

# تنظیمات مسیرهای ذخیره‌سازی
PDF_FILE_PATH = "The_New_Complete_Book_of_Food.pdf"
LANCE_DB_PATH = "food_knowledge.lance"
DB_PATH = "food_orders.db"

# پردازش فایل PDF با استفاده از LlamaParse
# پردازش فایل PDF با استفاده از LlamaParse
def parse_pdf(file_path):
    parser = LlamaParse(api_key=LLAMA_CLOUD_API_KEY)
    text_data = parser.load_data(file_path)
    text = "\n".join([doc.text for doc in text_data])
    return text


# تقسیم‌بندی متن به چانک‌ها
def split_text(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_text(text)
    return chunks


# ذخیره چانک‌ها در SQLite
def store_embeddings(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
    db_connection = lancedb.connect(LANCE_DB_PATH)
    table = db_connection.create_table("food_data", schema={"id": int, "text": str, "vector": list}, mode="overwrite")

    records = []
    for i, chunk in enumerate(chunks):
        embedding_vector = embeddings.embed_query(chunk)
        records.append({"id": i, "text": chunk, "vector": embedding_vector})

    table.add(records)
    return db_connection


# تابع برای جستجو در SQLite
def search_in_db(query):
    connection = sqlite3.connect(DB_PATH)
    cursor = connection.cursor()
    cursor.execute("SELECT content FROM food_data")
    results = cursor.fetchall()
    connection.close()
    matches = [row[0] for row in results if query.lower() in row[0].lower()]
    return matches


# تابع برای جستجو در اینترنت با استفاده از Tavily API
def search_online(query):
    url = "https://api.tavily.com/search"
    headers = {"Authorization": f"Bearer {TAVILY_API_KEY}"}
    params = {"query": query, "num_results": 3}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()["results"]
    return []


# تابع فیلتر کردن context نامرتبط
def filter_context(context):
    return context


# تابع اصلی برای پردازش سوالات کاربران
def process_user_query(question, model="gpt4o"):
    retrieved_docs = search_in_db(question)

    if retrieved_docs:
        context = "\n".join(retrieved_docs)
    else:
        search_results = search_online(question)
        context = "\n".join([result["snippet"] for result in search_results])

    context = filter_context(context)

    if not context:
        return "متأسفم، اطلاعاتی درباره این موضوع ندارم."

    llm = gpt4o_chat if model == "gpt4o" else gemini_chat
    response = llm(context)
    return response


# تعیین حوزه سوالات مجاز
def validate_question_scope(question):
    allowed_keywords = [
        "food", "meal", "nutrition", "diet", "restaurant", "cuisine", "dish", "recipe", "ingredients", "healthy eating",
        "calories", "cooking", "menu", "snack", "beverage", "gourmet", "organic food", "fast food", "fine dining", "vegan",
        "vegetarian", "protein", "carbohydrates", "fats", "vitamins", "minerals", "superfoods", "ethnic food", "street food"
    ]
    if not any(keyword in question.lower() for keyword in allowed_keywords):
        return "This question is outside the chatbot's domain."
    return None


# تابع پاسخگویی به سوالات کاربران
def answer_food_question(question, model="gpt4o"):
    validation_msg = validate_question_scope(question)
    if validation_msg:
        return validation_msg

    return process_user_query(question, model)



In [10]:

# پردازش و ذخیره‌سازی داده‌ها برای اولین بار
if __name__ == "__main__":
    PDF_FILE_PATH = "The_New_Complete_Book_of_Food.pdf"
    text = parse_pdf(PDF_FILE_PATH)


    print("پردازش و ذخیره‌سازی اطلاعات غذاها تکمیل شد!")




RuntimeError: Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.

In [ ]:
import os
import requests
import Levenshtein
import pyarrow as pa
import lancedb
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from llama_parse import LlamaParse
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# تنظیمات API
AVALAI_BASE_URL = "https://api.avalai.ir/v1"
GPT_MODEL_NAME = "gpt-4o-mini"
GEMINI_MODEL_NAME = "gemini-1.5-flash"
TAVILY_API_KEY = "your_tavily_api_key"
LLAMA_CLOUD_API_KEY = os.environ.get("LLAMA_CLOUD_API_KEY", "your_api_key_here")

# مقداردهی اولیه مدل‌ها
gpt4o_chat = ChatOpenAI(
    model=GPT_MODEL_NAME,
    base_url=AVALAI_BASE_URL,
    api_key=os.environ["AVALAI_API_KEY"]
)

gemini_chat = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME)

# تنظیمات مسیرهای ذخیره‌سازی
PDF_FILE_PATH = "The_New_Complete_Book_of_Food.pdf"
LANCE_DB_PATH = "./food_knowledge.lance"

# پردازش فایل PDF با استفاده از LlamaParse
def parse_pdf(file_path):
    parser = LlamaParse(api_key=LLAMA_CLOUD_API_KEY)
    text_data = parser.load_data(file_path)
    text = "\n".join([doc.text for doc in text_data])
    return text

# تقسیم‌بندی متن به چانک‌ها
def split_text(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_text(text)
    return chunks

# ذخیره چانک‌ها در LanceDB
def store_embeddings(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
    db_connection = lancedb.connect(LANCE_DB_PATH)
    schema = pa.schema([
        ("id", pa.int32()),
        ("text", pa.string()),
        ("vector", pa.list_(pa.float32()))
    ])
    table = db_connection.create_table("food_data", schema=schema, mode="overwrite")
    records = [{"id": i, "text": chunk, "vector": embeddings.embed_query(chunk)} for i, chunk in enumerate(chunks)]
    table.add(records)
    return db_connection

# جستجو در LanceDB
def search_in_db(query):
    db_connection = lancedb.connect(LANCE_DB_PATH)
    table = db_connection.open_table("food_data")
    results = table.search(query).limit(10).to_pandas()
    matches = results["text"].tolist()
    return matches

# تابع پاسخگویی به سوالات کاربران
def answer_food_question(question, model="gpt4o"):
    validation_msg = validate_question_scope(question)
    if validation_msg:
        return validation_msg
    return process_user_query(question, model)

# پردازش و ذخیره‌سازی داده‌ها برای اولین بار
if __name__ == "__main__":
    text = parse_pdf(PDF_FILE_PATH)
    chunks = split_text(text)
    store_embeddings(chunks)
    print("Processing and storing food information is complete!")
    
    while True:
        user_input = input("Ask your question about food (type 'exit' to quit): ")
        if user_input.lower() == "exit":
            break
        response = answer_food_question(user_input)
        print("Response:", response)
